# Practice #1: Learning how to use Jupyter Notebooks in Cuahsi HydroShare
## Justin Blaylock, 2023
This is an exercise where we learn to manage and run code in this Python environment.

## 1. Calling geemap

I am copying code from geemap 108 notebook to avoid the gee token setup.

In [1]:
import ee
import geemap
import geemap.colormaps as cm

## 2. Generate a map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## 3. Define a point of interest

Find lat/long of POI using the https://www.latlong.net/ website. In this case, coordinates are for Roy, Utah.

In [3]:
latitude = 41.167381
longitude= -112.033829

In [4]:
point = ee.Geometry.Point([longitude, latitude])

## 4. Retrieve satellite imagery: Landsat 8

Search for 'Landsat 8 Surface Corrected' here: https://developers.google.com/earth-engine/datasets/catalog/ and get the handle name (e.g. ee.ImageCollection("LANDSAT/LC08/C01/T1_SR"). Name the import 'landsat'.

In [5]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

In [6]:
# Note that we need to cast the result of first() to Image.
image = ee.Image(landsat

#    // Filter to get only images in the specified range.This range can be larger
    .filterDate('2013-07-01', '2013-07-31')  # choose a period of no snow

#    // Filter to get only images at the location of the point.This can be a polygon too.
    .filterBounds(point)

#    // Sort the collection by a metadata property.
    .sort('CLOUD_COVER_LAND')

#    // Get the first image out of this collection.
    .first()) #so the less cloudy image

# Get the timestamp and convert it to a date.
julian_date = ee.Date(image.get('system:time_start')).format('DD').getInfo();
year = ee.Date(image.get('system:time_start')).format('Y').getInfo();

print('Julian date: ', julian_date); # ee.Date
print('Year: ', year)

# What properties of this satellite are available?
# properties = image.propertyNames();
# print(properties.getInfo())

Julian date:  192
Year:  2013


In [7]:
# Print the image band names to the console. Check in GEE website what wavebands these results indicate.
band_names = image.bandNames()
print(band_names.getInfo())

['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


In [8]:
# select only reflectance bands
image = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
# The image requires the scale = 0.0001 as shown above.
image = image.multiply(2.75e-05).add(-0.2)

In [9]:
#// Add the image to the map, using the visualization parameters.
Map = geemap.Map(center=(latitude, longitude), zoom=8)
Map

Map(center=[41.167381, -112.033829], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

In [10]:
# // Center the map and display the image.
#Map.setCenter(-122.1899, 37.5010, 10); # San Francisco Bay
Map.addLayer(image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2']}, name='Landsat')
Map


Map(center=[41.167381, -112.033829], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…